# Обработка текстов

## Регулярные выражения

Часто возникает задача поиска в тексте каких-то элементов (например гиперссылок) или проверки введенных пользоватем данных. Для упрощения этих задач можно использовать регулярные выражения, реализация которых присутствует в стандартных библиотеках большей части популярных языков программирования, в том числе и в `Python`.

Наиболее типичные сценария использования:

* поиск паттерна в строке
* проверка строки на совпадение паттерну 
* сегментация строки по паттерну 
* замена паттерна в строке.

Паттерн описывается с помощью специального языка - регулярного выражения, в `Python` используются регулярные выражения со следующими базовыми наборами правил:

|           |                       |              |                               |
|-----------|-----------------------|--------------|-------------------------------|
| .         | любой символ          | \\d          | цифра                         |
| \\D       | не цифра              | \\s          | пробельный символ             |
| \\S       | не пробельный символ  | \\w          | буквенный символ              |
| \\W       | не буквенный символ   | ^            | начало строки                 |
| $         | конец строки          | \\b          | начало слова                  |
| \\B       | конец слова           | \[abc\]      | символ из перечисленных       |
| \[^abc\]  | кроме символов        | \[a\-zA\-Z\] | символы из интервалов         |
| X\|Y      | или                   |


Наборы символов и символьных классов можно объединять в группы, наподобие того как это происходит в обычных математических выражениях.  Для описания групп с различными целями существует следующие конструкции

|           |                      |           |                           |
|-----------|----------------------|-----------|---------------------------|
| \(X\)     | группа \(capturing\) | \(?:X\)   | группа \(non\-capturing\) |
| \(?=X\)   | предпросмотр         | \(?\!X\)  | негативный предпросмотр   |

Квантификатор после какого-то подвыражения (символа или группы) позволяет задать правила повторения этого подвыражения. Квантификатор может относиться более чем к одному символу в регулярном выражении только если это группа.  В `Python` определены жадные квантификаторы 

|          |                       |           |                          |
|----------|-----------------------|-----------|--------------------------|
| X?       | 0 или 1 повторение    | X\*       | $\geq0$ повторений       |
| X\+      | $\geq 1$ повторений   | X\{n\}    | ровно $n$ повторений     |
| X\{n,\}  | $\geq n$ повторений   | X\{n,m\}  | от $n$ до $m$ повторений |


и ленивые квантификаторы

|          |                       |           |                          |
|----------|-----------------------|-----------|--------------------------|
| X??      | 0 или 1 повторение    | X\*?      | $\geq0$ повторений       |
| X\+?     | $\geq 1$ повторений   | X\{n\}?   | ровно $n$ повторений     |
| X\{n,\}? | $\geq n$ повторений   | X\{n,m\}? | от $n$ до $m$ повторений |

Разница заключается в алгоритме сопоставления - жадные квантификаторы пытаются сопоставить как можно больше символов входной строки, ленивые - как можно меньше.



## Реализации регулярных выражений

### Формальные языки

**Формальный язык** — множество конечных слов над конечным алфавитом $\Sigma$. 
Пусть есть некоторое конечно множество символов $\Sigma$, тогда множество $L \in \Sigma^*$ есть формальный язык. 

Над формальными языками можно определить операции:

* $L_1 \cap L_2$
* $L_1 \cup L_2$
* $L_1 \setminus L_2$
* $L_1 \cdot L_2 $ - новый язык, в котором ко всем возможным словам из $L_1$ присоеденены справа слова из $L_2$
* $L^*$ - замыкание клини, $\{\epsilon\} \cup L \cup (L \cdot L) \cup (L \cdot L \cdot L) \cup \cdots$

Формальный язык над алфавитом $\Sigma$ является **регулярным**, если он принадлежит множеству языков $R \in \Sigma^*$:

* $\varnothing \in R$
* $\{\varepsilon\} \in R$
* $\forall a \in \Sigma: \{a\} \in R$
* $P \in R \land Q \in R \Rightarrow (P \cup Q) \in R$
* $P \in R \land Q \in R \Rightarrow (P \cdot Q) \in R$
* $P \in R \Rightarrow P^* \in R$

Любой регулярный язык может быть описан:

* детерменированным конечным автоматом
* недетерменированным конечным автоматом
* регулярным выражением
* регулярном грамматикой

**Конечные автоматы**

Конечный автомат это упорядоченная пятерка $A = (\Sigma, Q, q_0, F, \delta)$, где

* $\Sigma$ - входной алфавит
* $Q$ - множество состояний
* $q_0 \in Q$ - начальное состояние
* $F \subset Q$ - множество конечных состояний
* $\delta: (\Sigma \cup \varepsilon) \times Q \rightarrow 2^Q$ - функция перехода

В зависимости от определения функции перехода:

* недетерминированный конечный автомат с $\varepsilon$-переходами ($\varepsilon$-NFA)
$$\delta: (\Sigma \cup \varepsilon) \times Q \rightarrow 2^Q$$ 
* недетерминированный конечный автомат (NFA)
$$\delta: \Sigma \times Q \rightarrow 2^Q$$ 
* детерминированный конечный автомат (DFA)
$$\delta: \Sigma \times Q \rightarrow Q$$ 

Множество слов, которые принимаются конечным автоматом образуют регулярный язык. По любому $\varepsilon$-NFA можно построить эквивалентный DFA. В DFA можно минимизировать число состояний. 

Графическое отображение конечного автомата представлено на изображении

![конечный автомат](images/dfa.svg)


Существует два распространенных способа реализации регулярных выражений, применяемых в различных задачах (не считая гибридов и т.п.):

1. регулярное выражение -> $\varepsilon$-NFA -> DFA -> min-DFA
2. backtracking 

В стандартных библиотеках популярных языков программирования применяется backracking.

## Разновидности синтаксиса регулярных выражений

Исторически существует несколько возможных  синтаксисов языков для описания регулярных выражений

* POSIX RE[@posix-re-ref] (`., *, [ ], [^ ], \{ \}, \( \)`)
* POSIX ERE (`., *, +, ?, |, [ ], [^ ], { }, ( )`)
* PCRE[@pcre-ref] (стандартные библиотеки `Perl`, `Java`, `C#`, `Python`)

## Реализация в языке Python

В `Python` стандартные регулярные выражения определены в модуле `re`. Кроме того, существует специальный синтаксис задания строк для регулярных выражений - префикс `r`. В строках с этим префиксом не требуется специальным образом задавать специальные управляющие символы (например, `,`)


In [1]:
import re

Рекомендует сначала скомпилировать регулярное выражение с помощью функции `re.compile()`, в результате получится специальный объект, который можно использовать для выполнения основных задач

In [2]:
pattern = re.compile(r'a+a$')

Например, чтобы проверить совпадает ли паттерн, заданный регулярным выражением, со строкой, можно использовать метод `fullmatch`


In [3]:
print(pattern.fullmatch('aaaa'))

<_sre.SRE_Match object; span=(0, 4), match='aaaa'>


В данном случае строка `'aaaa'` соответствует паттерну `'a+a$'`, возвращается объект специального типа. Если соответствия нет, то возвращается `None`.

Второй возможный сценарий использования - поиск непересекающихся подстрок, которые соответствую паттерну. Введем паттерн - все слова из букв *a* которые заканчиваются на *b*.

In [4]:
pattern = re.compile(r'a+b')

Найдем все вхождения этого паттерна в строке:

In [5]:
s = 'Hello aaab world ab !'
pattern.findall(s)

['aaab', 'ab']

Ещё один сценарий - разделение строки на подстроки по паттерну. Это бывает полезным, когда нужно быстро разделить, например, предложения на слова, убрав все знаки препинания. 


In [6]:
pattern.split(s)

['Hello ', ' world ', ' !']

Следующий сценарий использования - замена вхождений паттерна в строке.


In [7]:
pattern.sub('e', s)

'Hello e world e !'


В качестве замены можно передать не только строку, но и `lambda`-функцию, которая будет вызываться на каждое совпадение с образцом. В данном случае мы добавляем к каждому совпадению букву *a*.

In [8]:
pattern.sub(lambda x: x.group(0) + 'a', s)

'Hello aaaba world aba !'


Приведем более сложный пример. В качестве паттерна опишем адрес электронной почты (упрощенный, данный паттерн не удовлетворяет спецификации).Дальше разобьем этот адрес на составляющие:

In [9]:
pattern = re.compile(r'(\w+)@(\w+)\.(\w{2,3})')

matcher = pattern.match('test@example.com')
if matcher:
    print(matcher.group(0))
    print(matcher.group(1))
    print(matcher.group(2))
    print(matcher.group(3))

test@example.com
test
example
com


Можно найти все вхождения электронной почты в тексте и напечатать только домены первого уровня:


In [10]:
t = 'test@example.com ssss test2@gmail.com'

pattern = re.compile(r'(\w+)@((\w+)\.(\w{2,3}))')
for m in pattern.finditer(t):
    print (m.group(2))

example.com
gmail.com


или имена пользователей:

In [11]:
s = 'test@example.com hello@mail.ru'
matchers = pattern.finditer(s)
for matcher in matchers:
    print (matcher.group(1))

test
hello


Обратим внимание на работу жадных и ленивых квантификаторов. В случае применения жадных результат может быть не тем, что ожидается


In [12]:
s = '<h1> text1 </h1>  <h2> text3 </h2>'
re.findall(r'<\w+>(?:.+)</\w+>', s)

['<h1> text1 </h1>  <h2> text3 </h2>']

В данном случае можно использовать ленивые

In [13]:
re.findall(r'<\w+>(?:.+?)</\w+>', s)

['<h1> text1 </h1>', '<h2> text3 </h2>']

Ещё один важный момент касается производительности движка регулярных выражений. Так как в `Python` регулярные выражения реализованы через механизм backtracking'а, то в некоторых случаях можно получить экспоненциальную сложность. 

In [14]:
re.findall(r'(a*a*)*c', 'a' * 10 + 'e')

[]

В модуле `re` есть недокументированный класс `Scanner`, с помощью которого можно реализовать лексический анализатор. `Scanner` будет искать вхождения паттернов в тексте и на каждое совпадение вызывать соответствующую функцию. В общем случае подобный код неэффективен, лексические анализаторы лучше реализовывать с помощью специальных инструментов - генераторов лексических анализаторов, которые обеспечат анализ за линейное время.

In [15]:
scanner = re.Scanner(
   [(r'(\w+)@(\w+)\.(\w{2,3})', lambda s, x: (x, 'email')),
    (r'[a-zA-Z]+', lambda s, x: (x, 'word')), 
    (r'\d+', lambda s, x: (x, 'digit')),    
    (r'\s+', lambda s, x: (x, 'whitespace')),
    (r'[.,;"!?:]', lambda s, x: (x, 'preposition')),
    ])

scanner.scan('hello, world 1234 test@example.com')

([('hello', 'word'),
  (',', 'preposition'),
  (' ', 'whitespace'),
  ('world', 'word'),
  (' ', 'whitespace'),
  ('1234', 'digit'),
  (' ', 'whitespace'),
  ('test@example.com', 'email')],
 '')

## Инструменты для обработки текстов

Привидем список некоторых полезных библиотек для обработки текстовых данных:

- ply(`Python`) - библиотека для написания лексических анализаторов на `Python`
- pyparsing] (`Python`) - библиотека для написания синтаксических анализаторов на `Python`
- lex, flex (`C`) - классические библиотеки - генераторы лексических анализаторов 
- jflex (`Java`) - библиотека для написания лексических анализаторов на `Java` 
- ANTLR(`Java`, `C++`, `Python`)
 

## NLTK
Natural Language Toolkit, библиотека для обработки естественных языков.Она создавалась для учебных целей, но тем не менее приобрела определенную популярность. Например в ней реализовано множество методов токенизации, которые можно использовать для повседневных задач и экспериментов.

In [16]:
import nltk
from nltk.tokenize import wordpunct_tokenize, word_tokenize

nltk.download('punkt')

(word_tokenize('Hello world 4.2.'), 
 word_tokenize('LA New-York'), 
 wordpunct_tokenize('Hello world 4.2!'))

[nltk_data] Downloading package punkt to /home/alex/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


(['Hello', 'world', '4.2', '.'],
 ['LA', 'New-York'],
 ['Hello', 'world', '4', '.', '2', '!'])

### Ply

Приведем лексического анализатора на `ply`. В данном случае анализатор описывается в классе, могут быть три вида токенов - слова, цифры и пробелы. Для каждого токена в тексте выозвращается необходимая информация - типа, длина смещение:

In [17]:
from ply.lex import lex, TOKEN

class Lexer:
    tokens = ( 'NUMBER', 'ID', 'WHITESPACE' )
    
    @TOKEN(r'\d{1,5}')
    def t_NUMBER(self, t):
        t.value = int(t.value)
        return t

    @TOKEN(r'\w+')
    def t_ID(self, t):
        return t

    @TOKEN(r'\s+')
    def t_WHITESPACE(self, t):
        pass

    def t_error(self, t):
        pass
    

__file__ = ''     # make `ply` happy

lexer = lex(object=Lexer())
lexer.input('123 abs 965')
for token in lexer:
    print(token)

LexToken(NUMBER,123,1,0)
LexToken(ID,'abs',1,4)
LexToken(NUMBER,965,1,8)


### Pyparsing

Другой пример `pyparsing`, с помощью которого можно обрабатывать более широкий класс формальных языков. С помощью специального DSL (domain-specific language, предметно-ориентированный язык) описывается грамматика. С помощью `pyparsing` можно обрабатывать коллекции в специфичных форматах, извлекать логи и так далее:

В данном примере грамматика описывает строку, которая начинается со слова, после которого идет двоеточие и набор чисел через запятую.

In [18]:
from pyparsing import Word, alphas, nums,  Literal, StringEnd, ZeroOrMore, Suppress, OneOrMore 

word = Word(alphas)
num = Word(nums)
sep = Suppress(OneOrMore(','))
col = Suppress(':')

s = word + col + num + ZeroOrMore(sep + num) + StringEnd()
        
s.parseString('hello: 1, 22, 3')

(['hello', '1', '22', '3'], {})

Здесь более сложный пример, грамматика описывает правильные скобочные записи.


In [19]:
from pyparsing import Literal, Forward, StringEnd, OneOrMore, Empty

br_o = Literal('(')
br_c = Literal(')')

braces = Forward()
braces << OneOrMore(br_o + (braces | Empty() ) + br_c)
start = braces + StringEnd()
        
start.parseString('(())()()')

(['(', '(', ')', ')', '(', ')', '(', ')'], {})

И, наконец, простейшие математические выражения с приоритетом операций. Сначала можно определить классы, которые будут узлами дерева выражения


In [20]:
from pyparsing import Word, Literal, Or, nums, Forward, StringEnd
from operator import mul, truediv, add, sub

class NumNode(object):
    def __init__(self, t):
        self.num = float(t[0])        
    def calc(self):
        return self.num          
    def __repr__(self):
        return 'Num(%s)' % self.num
        
class OpNode(object):
    def __init__(self, t):               
        self.left = t[0]
        self.op = { '-' : sub, '+' : add, '/' : truediv, '*' : mul }[t[1]]
        self.right = t[2]       
    def calc(self):
        return self.op(self.left.calc(), self.right.calc())        
    def __repr__(self):
        return 'Op(%s, %s, %s)' % (self.left, self.op, self.right)

Опишем грамматику

In [21]:
plus = Literal('+')
minus = Literal('-')
div = Literal('/')
mult = Literal('*')
        
factor = Word(nums).setParseAction(NumNode)

term = Forward()
term << (( factor + (mult | div) + term ).setParseAction(OpNode) | factor )        

expr = Forward()
expr << ((term + (plus | minus) + expr).setParseAction(OpNode) | term )

start = expr + StringEnd()


In [22]:
tree = start.parseString('2 * 4 + 6 * 7')[0]
print(tree)
print(tree.calc())

Op(Op(Num(2.0), <built-in function mul>, Num(4.0)), <built-in function add>, Op(Num(6.0), <built-in function mul>, Num(7.0)))
50.0


### JFlex

In [23]:
cat code/jflex/src/main/jflex/TestScanner.jflex

package ru.spbu.apmath.pt.lexer;

import java.io.IOException;
import ru.spbu.apmath.pt.lexer.TokenTypes;


%%

%class TestScanner
%public
%unicode
%ignorecase
%type TokenTypes
%function getNextToken
%pack
%char

%{


public String getText() {
    return yytext();
}

public int getCurPosistion() {
    return yylength();
}

%}

ALPHA = [a-zA-Z]
DIGIT = [0-9]


COMMENT   = "/*"  ~"*/"

IDENTIFIER = {ALPHA} ({ALPHA} | {DIGIT})*

INTEGER = 0 | [1-9][0-9]*


%%

<YYINITIAL> {
    {IDENTIFIER} { return TokenTypes.TOKEN_IDENTIFIER; }
    {INTEGER} { return TokenTypes.TOKEN_INTEGER; }
    {COMMENT} {  }
    . {    }
}


In [24]:
cat code/jflex/src/main/java/ru/spbu/apmath/pt/lexer/TestScanner.java

/* The following code was generated by JFlex 1.4.3 on 14.10.13 21:58 */

package ru.spbu.apmath.pt.lexer;

import java.io.IOException;
import ru.spbu.apmath.pt.lexer.TokenTypes;



/**
 * This class is a scanner generated by 
 * <a href="http://www.jflex.de/">JFlex</a> 1.4.3
 * on 14.10.13 21:58 from the specification file
 * <tt>C:/Users/alms/home/work/asp/compilers/lexer/src/main/jflex/TestScanner.jflex</tt>
 */
public class TestScanner {

  /** This character denotes the end of file */
  public static final int YYEOF = -1;

  /** initial size of the lookahead buffer */
  private static final int ZZ_BUFFERSIZE = 16384;

  /** lexical states */
  public static final int YYINITIAL = 0;

  /**
   * ZZ_LEXSTATE[l] is the state in the DFA for the lexical state l
   * ZZ_LEXSTATE[l+1] is the state in the DFA for the lexical state l
   *                  at the beginning of a line
   * l is of the form l = 2*k, k a non negative integer
   */
  private static final int ZZ_LEXSTATE[] = { 
   

In [22]:
cat code/jflex/src/main/java/ru/spbu/apmath/pt/lexer/LexerExample.java

package ru.spbu.apmath.pt.lexer;

import java.io.StringReader;

public class LexerExample {

    public static void main(String[] args) throws Exception {
        //Разбиваем строку на токены (комментарии игнорируем).
        final String exampleText = "hello world 2012 /* comment */ ";

        final TestScanner scanner = new TestScanner(new StringReader(exampleText));

        TokenTypes token = null;
        while ((token = scanner.getNextToken()) != null) {
            System.out.println(token + " " + scanner.getText() + " " + scanner.getCurPosistion());
        }
    }
}
